<img src="../images/AzPTravel_PPM.png">

## GPM Pipeline Output maker

#### This script
- loads the transformed parquet file
- creates the file {parquet}.localcur.csv
- makes the Euro conversions
- outputs {parquet}.euroconv.csv


#### Parameters section

In [1]:
commit = ""
# give a reason for the run

percode = "20XX.QX"
# Data Collection Code, this controls file paths and output names

run_type = 1
#run_type =  0 - lite run with no reporting, not recommended.
#run_type =  1 - lite run with normal reporting, default setting.
#run_type =  2 - Heavy run with full reporting, available for audits and troubleshooting.

specialchars = "-GTHtest"
# optional - add up to a 12 character code in order to mark your instance record .ipynb

from datetime import datetime
inst_datetime = datetime.now().strftime("%m%d%Y%H%M%S")
# a single datetime stamp for the full instance run


In [2]:
# Parameters
run_control = 1
percode = "2021.Q1"
commit_message = "Demo Test"
inst_datetime = "05282021141936"


### Set Run Control

##### 0 - lite run with no reporting, not recommended.
##### 1 - lite run with normal reporting, default setting.
##### 2 - Heavy run with full reporting, available for audits and troubleshooting.

In [3]:
run_control = 0 # in development mode

#### Packages used

In [4]:
import sys
import os
import pandas as pd

In [5]:
default_DC = "2021.Q1"

try:
    if sys.argv[1] == "-f":
        percode = default_DC
    else:
        percode = sys.argv[1]

except IndexError:
    percode = default_DC
except NameError:
    percode = default_DC

#### Convert paths and files to variables.
#### Read data into pd DataFrames.
#### Make paths for the live sources.

In [6]:
rt_path = os.path.join('\\\\hecate\\Insurance_US\\Product Development\\Product Management\\Global PPM\\Reporting\\Data Collection\\Production',percode)





#### File names

In [7]:
## Input files
tempfile = os.path.join(rt_path, f'{percode}.localcur.parquet')
gppm_file = os.path.join(rt_path, f'GPPM_Input_{str(percode)}.xlsx')

## Output files
local_file = f'{percode}.localcur.csv'
euro_file = f'{percode}.euroconv.csv'



#### Read source

In [8]:
mo_input = pd.read_parquet(tempfile, engine='pyarrow')


"""reads in the input for the Euro conversion"""
euro = pd.read_excel(gppm_file, sheet_name="Euro_Rates")


#### Output local currency csv

In [9]:
mo_input.to_csv(os.path.join(rt_path,local_file), ',', index=False, encoding='utf-8-sig')

#### Convert all currency to Euro


In [10]:

cflds = ['written_revenues_net_of_taxes', 'written_revenues', 'earned_revenues_net_of_taxes',
         'earned_revenues', 'earned_base_commissions', 'earned_over_commissions', 'upfront_cash_payments',
         'total_compensation', 'paid_claims', 'ocr__ibnr', 'actual_incurred_losses',
         'internal_variable_costs', 'az_tech_fee', 'internal_fixed_costs',
         'hq_fees', 'total_expenses', 'risk_premium', 'profit_or_loss', 'cm_hq_view', 'cm_bu_view', ]

alt_cflds = ['written_revenues_net_of_taxes', 'written_revenues', 'earned_revenues_net_of_taxes',
         'earned_revenues', 'earned_base_commissions', 'earned_over-commissions', 'upfront_cash_payments',
         'total_compensation', 'paid_claims', 'ocr_+_ibnr','actual_incurred_losses_(paid_+_ocr_+_ibnr)',
         'internal_variable_costs_(excl._az_tech_fee)', 'az_tech_fee', 'internal_fixed_costs_(excl._hq_fees)',
         'hq_fees', 'total_expenses', 'risk_premium', 'profit_or_loss', 'contribution_margin_-_hq_view', 'contribution_margin_-_bu_view', ]




#### Make a copy for debug purposes

In [11]:
trandata = mo_input.copy()

#### Convert Currency Below

In [12]:
# create DataFrame of unique currencies with rates

fx_uni = euro[['CURR_CODE','RATE']]
fx_uni = fx_uni.drop_duplicates()

In [13]:
# add euro conversion rates to trandata DataFrame as column 'eur_rate'

eurdata = pd.merge(trandata['currency'], fx_uni, left_on='currency', right_on='CURR_CODE', how='left')

trandata['eur_rate'] = eurdata['RATE']

#### Convert currency fields

In [14]:
try:
    trandata[cflds] = trandata[cflds].divide(trandata['eur_rate'], axis="index").fillna(0)

except KeyError:
    trandata[alt_cflds] = trandata[alt_cflds].divide(trandata['eur_rate'], axis="index").fillna(0)

    

#### Output to {percode}.euroconv.csv

In [15]:
trandata.to_csv( os.path.join(   rt_path , euro_file) , ',', index=False, encoding='utf-8-sig')


In [16]:
#### Store the DataFrame for other notebooks to use and open the file location for the user to view the new files

In [17]:
mo_df = trandata

%store mo_df


os.startfile( rt_path   )



Stored 'mo_df' (DataFrame)
